In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import imdb
import numpy as np

In [4]:
num_words = 10000
maxlen = 200

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

In [14]:
# Pad sequences to the same length
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Build the Bidirectional LSTM model
model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=128, input_length=maxlen))  # Embedding layer
model.add(Bidirectional(LSTM(64, return_sequences=False)))  # Bidirectional LSTM layer
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=64)

Epoch 1/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 64s 357ms/step - accuracy: 0.6086 - loss: 0.6360 - val_accuracy: 0.8406 - val_loss: 0.3693
Epoch 2/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 55s 342ms/step - accuracy: 0.8727 - loss: 0.3118 - val_accuracy: 0.8430 - val_loss: 0.3549
Epoch 3/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 43s 267ms/step - accuracy: 0.9344 - loss: 0.1869 - val_accuracy: 0.8621 - val_loss: 0.3343
Epoch 4/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 47s 297ms/step - accuracy: 0.9597 - loss: 0.1299 - val_accuracy: 0.8547 - val_loss: 0.4281
Epoch 5/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 49s 305ms/step - accuracy: 0.9699 - loss: 0.0955 - val_accuracy: 0.8559 - val_loss: 0.5001


In [16]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")

# Function to decode the integer-encoded reviews back to words
def decode_review(encoded_review):
    word_index = imdb.get_word_index()
    reverse_word_index = {value: key for key, value in word_index.items()}
    decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review if i > 2])  # Offset by 3 for padding
    return decoded_review

# Function to predict sentiment for a given review
def predict_sentiment(review):
    word_index = imdb.get_word_index()
    encoded_review = [word_index.get(word, 0) + 3 for word in review.split()]  # Add offset by 3
    padded_review = pad_sequences([encoded_review], maxlen=maxlen)
    prediction = model.predict(padded_review, verbose=0)  # Disable verbose output
    sentiment = "Positive" if prediction[0][0] >= 0.5 else "Negative"
    return sentiment

# Output sample sentiments from the test set
print("\nSample Sentiments:")
for i in range(5):  # Print 5 sample sentiments
    review = decode_review(X_test[i])
    sentiment = predict_sentiment(review)
    print(f"Review: {review}\nSentiment: {sentiment}\n")
    print("-" *50)

782/782 ━━━━━━━━━━━━━━━━━━━━ 45s 57ms/step - accuracy: 0.8424 - loss: 0.5676
Test accuracy: 84.20%

Sample Sentiments:
Review: please give this one a miss br br and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite so all you madison fans give this a miss
Sentiment: Negative

--------------------------------------------------
Review: psychological it's very interesting that robert altman directed this considering the style and structure of his other films still the trademark altman audio style is evident here and there i think what really makes this film work is the brilliant performance by sandy dennis it's definitely one of her darker characters but she plays it so perfectly and convincingly that it's scary michael burns does a good job as the mute young man regular altman player michael murph

In [12]:
review="What a worst movie , I will not recommend to any one "
print(predict_sentiment(review))

Negative


In [13]:
review="Awesome and amazing movie , best experince"
print(predict_sentiment(review))

Positive
